## Download and install dependencies

In [ ]:
import shutil
import os

# Create the directory, including any necessary parent directories
os.makedirs('/kaggle/working/lmms-finetune', exist_ok=True)

%cd /kaggle/working

!git clone https://github.com/thisisAranya/lmms-finetune.git

%cd /kaggle/working/lmms-finetune

In [ ]:
!python -m pip install -r requirements.txt

## Load The Model

In [ ]:
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)

original_model_id = "Original Model Path"

# Give the adapter's path or the path of the tuned and merged model
model_id = "Tuned Adapter Path" or "Tuned & Merged Model Path" 

model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True,
)

processor = AutoProcessor.from_pretrained(original_model_id)

model.to('cuda')

## See the Output

In [ ]:
import os
import csv
import torch
from PIL import Image
from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig


# Path to the folder containing the images
image_folder = 'Image Folder Path'  # Replace with the actual path to your images folder

# Prepare the CSV file to store results
output_csv = 'Output CSV Path'

# Open CSV file for writing
with open(output_csv, 'w', newline='') as csvfile:
    fieldnames = ['image', 'output']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Iterate over all the images in the folder
    for image_filename in os.listdir(image_folder):
        image_file = os.path.join(image_folder, image_filename)
        
        try:
            # Load the image
            raw_image = Image.open(image_file)

            # Prepare conversation prompt
            conversation = [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "What is the name of this disease?"},
                        {"type": "image"},
                    ],
                },
            ]
            prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

            # Prepare the input
            inputs = processor(images=raw_image, text=prompt, return_tensors='pt').to('cuda', torch.float16)

            # Generate output
            output = model.generate(**inputs, max_new_tokens=128, do_sample=False)
            decoded_output = processor.decode(output[0][0:], skip_special_tokens=True)

            # Write the results to CSV
            writer.writerow({'image': image_filename, 'output': decoded_output})
            print(f"Processed image: {image_filename}")
            print(f"output: {decoded_output}")
            print("\n")

        except Exception as e:
            print(f"Error processing image {image_filename}: {e}")

print(f"Results have been saved to {output_csv}")